In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd


json_keyfile = "/Users/tanishqa/Downloads/vader-sentiment-analysis-e6711036e61c.json"

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)

client = gspread.authorize(creds)
sheet = client.open("Copy of Spreadsheet Data w/ Sentiment Predictions").sheet1
data = sheet.get_all_records()
df = pd.DataFrame(data)

print(df.head())  # Show first few rows


/Users/tanishqa/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


  Column1       Dates                                              Links  \
0          2023-04-25  https://www.reddit.com/r/wallstreetbets/commen...   
1       1  2023-01-24  https://www.reddit.com/r/wallstreetbets/commen...   
2       1  2023-10-29  https://www.reddit.com/r/wallstreetbets/commen...   
3       8  2023-07-25  https://www.reddit.com/r/wallstreetbets/commen...   
4       2  2023-04-27  https://www.reddit.com/r/wallstreetbets/commen...   

                                                text     label  
0  Bro bro brosuf bitch as mother fucking regard ...  0.010114  
1  FUCKING MORONS!!! MSFT Misses and it goes up. ...   0.01012  
2  Seriously I don’t know why any sane company wo...  0.010152  
3  Where is $30k $MSFT calls guy? I want kick his...  0.010177  
4  Ugh I’m a fucking dumbass\n\n50 aapl 170 calls...  0.010268  


In [4]:
print(df.columns)

Index(['Column1', 'Dates', 'Links', 'text', 'label'], dtype='object')


In [9]:
from transformers import DistilBertTokenizer
from sklearn.model_selection import train_test_split


tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)


df_tokenized = df.apply(lambda row: tokenizer(row['text'], padding="max_length", truncation=True), axis=1)

#Split training data into 80% train 20% test
train_data, test_data = train_test_split(df, test_size=0.2)


print(train_data.head())


      Column1       Dates                                              Links  \
18047       1  2023-02-01  https://www.reddit.com/r/wallstreetbets/commen...   
22994       3  2023-11-20  https://www.reddit.com/r/wallstreetbets/commen...   
12403       2  2023-05-01  https://www.reddit.com/r/wallstreetbets/commen...   
16658       4  2023-01-02  https://www.reddit.com/r/wallstreetbets/commen...   
20689       1  2023-02-07  https://www.reddit.com/r/wallstreetbets/commen...   

                                                    text     label  
18047  Hess not trading, he’s just holding long term ...  0.576233  
22994               msft is going to moon on market open  0.844681  
12403  But it is profitable. Only nvidia stock is exp...  0.492164  
16658                                               MSFT  0.547687  
20689  &gt;MICROSOFT'S AI POWERED EDGE CAN ANALAYZE F...  0.683417  


In [11]:
import torch


train_encodings = tokenizer(list(train_data['text']), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True, max_length=512)


train_labels = torch.tensor(train_data['label'].values)  # Replace with your sentiment label column
test_labels = torch.tensor(test_data['sentiment_column'].values)  # Replace with your sentiment label column


train_inputs = torch.tensor(train_encodings['input_ids'])
test_inputs = torch.tensor(test_encodings['input_ids'])


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [12]:

print(train_data['label'].unique())  # List unique values to see if anything is not numeric


[0.5762327015 0.8446812332 0.4921642989 ... 0.07301665284 0.5391993523
 0.01821408095]


In [13]:

train_labels = torch.tensor(train_data['label'].values, dtype=torch.float)
test_labels = torch.tensor(test_data['label'].values, dtype=torch.float)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [14]:

train_data['label'] = pd.to_numeric(train_data['label'], errors='coerce')
test_data['label'] = pd.to_numeric(test_data['label'], errors='coerce')


train_data = train_data.dropna(subset=['label'])
test_data = test_data.dropna(subset=['label'])

train_labels = torch.tensor(train_data['label'].values, dtype=torch.float)
test_labels = torch.tensor(test_data['label'].values, dtype=torch.float)


print(train_labels.dtype)
print(test_labels.dtype)


torch.float32
torch.float32


In [15]:
print(train_data['label'].head())


18047    0.576233
22994    0.844681
12403    0.492164
16658    0.547687
20689    0.683417
Name: label, dtype: float64


In [16]:
from transformers import DistilBertTokenizer


tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


train_encodings = tokenizer(list(train_data['text']), truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True, max_length=512, return_tensors="pt")



In [17]:
# Convert tokenized data into PyTorch tensors
train_inputs = torch.tensor(train_encodings['input_ids'])
test_inputs = torch.tensor(test_encodings['input_ids'])


/var/folders/t4/hm0qvm6d4y5cnzj9d_ll99lc0000gn/T/ipykernel_58377/3621506204.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_encodings['input_ids'])
/var/folders/t4/hm0qvm6d4y5cnzj9d_ll99lc0000gn/T/ipykernel_58377/3621506204.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_inputs = torch.tensor(test_encodings['input_ids'])


In [18]:
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, inputs, labels, attention_mask):
        self.inputs = inputs
        self.labels = labels
        self.attention_mask = attention_mask

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Create datasets
train_dataset = SentimentDataset(train_encodings['input_ids'], train_labels, train_encodings['attention_mask'])
test_dataset = SentimentDataset(test_encodings['input_ids'], test_labels, test_encodings['attention_mask'])


In [19]:
from torch.utils.data import DataLoader

# Create DataLoader for batching
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)



In [20]:
from transformers import DistilBertForSequenceClassification

# Initialize the DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)  # num_labels=1 for regression


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
device = torch.device("cpu")  # Force CPU usage
model.to(device)



DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
import torch.optim as optim
from torch import nn
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=5e-5)


# Define loss function (if not already included in the model)
loss_fn = torch.nn.MSELoss()  # Use MSELoss since labels are continuous values (float)

# Move model to the correct device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example of a simple training loop with attention_mask
epochs = 1
for epoch in range(epochs):
    print(f"Starting epoch {epoch+1}...")  # Debug: Print when each epoch starts
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()

        # Get inputs, attention_mask, and labels from the batch
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels'] 
# Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    # Print loss for this epoch
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_dataloader)}")

Starting epoch 1...
